<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Augmentation

Data augmentation is a technique that increases teh fiversity of your training set by applying random (but realistic) transformations such as image rotation. TF enables a couple of approach to implement data augmentation: `keras preprocessing layers` and `tf.image`.